# Deutsch Algorithm implementation

In [1]:
%matplotlib inline
import qutip as qt
from qutip import *
from pylab import *
from scipy import *
from scipy.integrate import quad
from scipy.linalg import expm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import string
import time

In [2]:
# Define the Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

# Define the Hamiltonian parameters
Ec = 1   # charging energy
Ej = 0.005   # Josephson energy
ng1 = 0.5  # gate charge for qubit 1
ng2 = 0.5  # gate charge for qubit 2
t = 0.15    # evolution time
Ecc = -0.005 # Columb charging 

# Define the Hamiltonian matrices
n1 = np.diag([0, 1])
n2 = np.diag([0, 1])
H1 = Ec * np.kron(sigma_z, (n1 - ng1) ** 2) - 0.5 * Ej * np.kron(sigma_x, np.eye(2))
H2 = Ec * np.kron((n2 - ng2) ** 2, sigma_z) - 0.5 * Ej * np.kron(np.eye(2), sigma_x)
H12 = Ecc * np.kron(sigma_z, sigma_z)

# Construct the total Hamiltonian
H = H1 + H2 + H12

# Define the ideal CNOT gate matrix
CNOT_ideal = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

# Define the initial state vector
psi0 = np.array([1, 0, 0, 0])   # |00> state

# Compute the time-evolution operator
U = expm(-1j * H * t) # we set hbar = 1 

# Apply the ideal CNOT gate to the initial state vector
psi_final = np.dot(CNOT_ideal, np.dot(U, psi0))

# Print the final state vector
print(psi_final)

[ 0.99586903-7.41132829e-02j -0.00139091-3.70557703e-02j
  0.00140479+3.50934377e-07j -0.00139091-3.70557703e-02j]


In [3]:
# Define the actual gate matrix
U_actual = np.reshape(psi_final, (2, 2))
U_actual = np.kron(U_actual, np.eye(2))

U_actual.shape

(4, 4)

In [4]:
U_actual

array([[ 0.99586903-7.41132829e-02j,  0.        +0.00000000e+00j,
        -0.00139091-3.70557703e-02j,  0.        -0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.99586903-7.41132829e-02j,
         0.        -0.00000000e+00j, -0.00139091-3.70557703e-02j],
       [ 0.00140479+3.50934377e-07j,  0.        +0.00000000e+00j,
        -0.00139091-3.70557703e-02j,  0.        -0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.00140479+3.50934377e-07j,
         0.        -0.00000000e+00j, -0.00139091-3.70557703e-02j]])

In [5]:
import numpy as np
from scipy.linalg import norm 

H_matrix=1/np.sqrt(2)*np.array([[1, 1],
                                [1,-1]])

# CNOT_matrix=np.array([[1,0,0,0],
#                       [0,1,0,0],
#                       [0,0,0,1],
#                       [0,0,1,0]])

# Using actual gate matrix instead of CNOT
CNOT_matrix = U_actual
CNOT_tensor=np.reshape(CNOT_matrix, (2,2,2,2))

In [6]:
class Reg: 
    def __init__(self,n):
        self.n=n
        self.psi=np.zeros((2,)*n) 
        self.psi[(0,)*n]=1
        
def H(i,reg): 
    reg.psi=np.tensordot(H_matrix,reg.psi,(1,i)) 
    reg.psi=np.moveaxis(reg.psi,0,i)

def CNOT(control, target, reg):
    reg.psi=np.tensordot(CNOT_tensor, reg.psi, ((2,3),(control, target))) 
    reg.psi=np.moveaxis(reg.psi,(0,1),(control,target))   

def measure(i,reg): 
    projectors=[ np.array([[1,0],[0,0]]), np.array([[0,0],[0,1]]) ] 
    
    def project(i,j,reg): 
        projected=np.tensordot(projectors[j],reg.psi,(1,i))
        return np.moveaxis(projected,0,i)
    
    projected=project(i,0,reg) 
    norm_projected=norm(projected.flatten()) 
    if np.random.random()<norm_projected**2: 
        reg.psi=projected/norm_projected
        return 0
    else:
        projected=project(i,1,reg)
        reg.psi=projected/norm(projected)
        return 1

In [7]:
# State 0 for both registers
reg=Reg(2)
reg.psi

array([[1., 0.],
       [0., 0.]])

In [8]:
# Apply Hadamard transformation to register - first time
for i in range(reg.n):
    H(i,reg)

reg.psi

array([[0.5, 0.5],
       [0.5, 0.5]])

In [9]:
# Assuming a constant function f = [1,0,1,0]
# Appplying the f-controlled phase shift
func_x =np.array([[1,0],[1,0]])


func_x = np.where(func_x == 0, -1, func_x)

state_funcx = np.dot(reg.psi, func_x)

reg.psi = state_funcx

reg.psi

array([[ 1., -1.],
       [ 1., -1.]])

In [10]:
# Apply Hadamard transformation again to register - Final time
for i in range(reg.n):
    H(i,reg)
reg.psi

array([[1.24465706e-18, 2.00000000e+00],
       [1.19558417e-33, 6.50353591e-17]])

In [11]:
# measure the final state of the bit 0
print('measure(0,reg)',measure(0,reg))

print('reg psi_0 ',reg.psi)
# measure the final state of the bit 0
# print('measure(1,reg)',measure(1,reg))

measure(0,reg) 0
reg psi_0  [[6.22328532e-19 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00]]


In [12]:
# measure the final state of the bit 0
#print('measure(0,reg)',measure(0,reg))

print('reg psi_1 ',reg.psi)
# measure the final state of the bit 0
print('measure(1,reg)',measure(1,reg))

reg psi_1  [[6.22328532e-19 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00]]
measure(1,reg) 1


In [13]:
if measure(0,reg) > 0:  # f(x) is balanced
    print("The function of the first qubit is balanced.")
else:  # f(x) is constant
    print("The function of the first qubit is constant.")


The function of the first qubit is constant.


In [14]:
if measure(1,reg) > 0:  # f(x) is balanced
    print("The function of the second qubit is balanced.")
else:  # f(x) is constant
    print("The function of the second qubit is constant.")

The function of the second qubit is balanced.
